In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Vamos a implementar Bert usando Keras y tensorflow 2.0.0  

Primero tenemos que definir la capa de MultiHeadAttention. Todavía no fue incluída como capa propiamente dicha en Keras, pero hay una implementación oficial en la página de keras que vamos a explicar:

In [2]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        
        ## que es este super?
        ## super() te permite acceeder a los metodos de la super clase de la cual
        ## la subclase está heredando. En este caso, estas herendando de Layers.
        
        ## definimos algunos parametros: cuantas cabezas va a tener self attention 
        ## y la dimensionalidad del embedding
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        ## la dimensionalidad del embedding tiene que ser divisible por el numero de cabezas.
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        
        ## cuantas dimensiones va a tener cada cabeza:
        self.projection_dim = embed_dim // num_heads
        
        ## y definimos las capas de key, query y value.
        ## son simplemente capas lineales
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        
        ## y definimos la capa con la que combinamos las cabezas
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        
        ## vamos a hacer lo que vimos en las diapos: 
        ## primero el producto entre el query y la transpuesta de key
        score = tf.matmul(query, key, transpose_b=True)
        
        ## tomamos la cantidad de columnas de key y la usamos para escalar el score
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        
        ## aplicamos la softmax para tener los attention weights y multiplicamos con values
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        ## vamos a armar la division en cabezas. 
        ## se va a entender mejor en el siguiente bloque de codigo
        ## por ahora es solamente la forma en la que 
        ## reacomodamos los datos para armar las cabezas
        
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        
        ## vamos a ir dejando anotado la forma de cada tensor para que sea mas facil de seguir
        
        ## cuando empezamos:
        ## x.shape = [batch_size, seq_len, embedding_dim]
        ## es decir, tenemos batch_size casos, con seq_len cantidad de vectores
        ## (1 por cada token), y cada vector tiene embedding_dim dimensiones
        
        batch_size = tf.shape(inputs)[0]
        
        ## al costado de cada linea vamos a marcar el tamaño que queda el tensor:
        ## para query, key y value vamos a tener tensores con batch_size casos, 
        ## con un largo de seq_len y la ultima dimension pasa a ser embed_dim, 
        ## que es el tamaño que definimos arriba en las Dense layers.
        ## Es decir, solo modificamos la 3er dimension del tensor.
        ## va a ser igual para query, key y value
        query = self.query_dense(inputs)  ## (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        
        ## ahora vamos a separar en las cabezas:
        query = self.separate_heads(query, batch_size)  
        ## nos va a quedar:
        ## (batch_size, num_heads, seq_len, projection_dim)
        
        ## repetimos:
        key = self.separate_heads(key, batch_size)  
        value = self.separate_heads(value, batch_size)  
        
                
        ## ahora vamos a usar la capa atencional y vamos a obtener la salida y los pesos
        attention, weights = self.attention(query, key, value)
        
        ## y reorganizamos la salida de tal manera que me quede:
        ## (batch_size, seq_len, num_heads, projection_dim)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])  
        
        
        ## y ahora concatenamos las cabezas!
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))  
        ## el tamaño ahora es (batch_size, seq_len, embed_dim)
        ## que s el mismo que teníamos al principio en query, key y value
        
        ## y metemos la ultima capa densa:
        output = self.combine_heads(concat_attention)  
        ## nos queda: (batch_size, seq_len, embed_dim)
        return output

Ahora que ya definimos la capa de multihead attention, pasamos a definir un bloque de transformer:

In [3]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        
        ## definimos una capa de multihead attention
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        
        ## definimos una secuencia de capa lineal, relu (en el paper original usan gelu)
        ## y capa lineal. Como el input van a ser batches de 2d, esto se va a aplicar
        ## como una time distributed layer, es decir, se va a aplicar a cada timestep
        
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        
        ## definimos capas de layernormalization y de dropout
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        ## aplicamos atencion y dropout
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        ## hacemos una residual connection y layer normalization
        out1 = self.layernorm1(inputs + attn_output)
        
        ## tenemos la position-wise feed forward
        ffn_output = self.ffn(out1)
        
        ## y repetimos dropout, residual connection y layer normalization
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Ya casi estamos. Solamente nos hace falta la capa de embeddings.  
Necesitamos el embedding del token y el embedding de la posición:

In [4]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        
        ## instanciamos dos capas de Embedding
        ## una de vocabulario x dimension del embedding
        ## otra de largo de secuencia x dimension del embedding
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim)

    def call(self, x):
        ## generamos un tensor de posicion para cada token
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        
        ## usamos la posicion para el embedding
        positions = self.pos_emb(positions)
        
        ## tomamos el embedding de cada token
        x = self.token_emb(x)
        
        ## y los sumamos
        return x + positions

Ya tenemos los ingredientos básicos para armar un transformer. Vamos a unir todo. Vamos a hacer algo que podamos entrenar en clase, así que en vez de hacer el tamaño total, vamos a hacer uno "mini":

In [5]:
vocab_size = 20000  # Vamos a considerar solamente 20k palabras
maxlen = 200  #  Solo considerar las primeras 200 palabras

embed_dim = 32  # Tamaño del embedding
num_heads = 2  # Numero de attention heads
ff_dim = 32  # Tamaño de la hidden layer adentro del transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)

x = transformer_block(x)

## la salida del bloque la vamos a poolear con un global average pooling:
## si tenemos (batch, seq, embed), hacemos el promedio y nos queda (batch, embed)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

Ahora si! podemos usarlo para lo que queramos. Por ahora vayamos con algo sencillo: clasificacion de texto

In [6]:
## me traigo el dataset de imdb (nota: acá no está con sentencepiece)
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

25000 Training sequences
25000 Validation sequences


In [7]:
## y voy a entrenarlo con solo 1 epoch a ver como da!

In [8]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=1, validation_data=(x_val, y_val)
)

Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 103s 4ms/sample - loss: 0.3756 - accuracy: 0.8268 - val_loss: 0.2906 - val_accuracy: 0.8766
